In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
import scipy.stats as stats
import tkinter as tk
import matplotlib

# 使用TkAgg后端
matplotlib.use('TkAgg')

class InteractiveAutoDistribution:
    def __init__(self, root):
        self.root = root
        self.root.title("正态分布自动可视化")
        self.root.geometry("900x700")

        # 创建控制面板
        control_frame = tk.Frame(root, padx=10, pady=10)
        control_frame.pack(side=tk.TOP, fill=tk.X)

        # 创建图形框架
        fig_frame = tk.Frame(root)
        fig_frame.pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        # 初始化参数
        self.mu = 0
        self.sigma = 1.0
        self.colour = 'blue'

        # 自动动画参数
        self.auto_mu = -5
        self.auto_mu_direction = 0.1
        self.auto_sigma = 0.5
        self.auto_sigma_direction = 0.02

        # 创建滑块和选择器
        self.create_controls(control_frame)

        # 创建图形
        self.fig, (self.ax1, self.ax2) = plt.subplots(2, 1, figsize=(8, 8))
        self.fig.patch.set_facecolor('#eeefef')

        # 将图形嵌入到Tkinter窗口
        self.canvas = FigureCanvasTkAgg(self.fig, master=fig_frame)
        self.canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)

        # 添加工具栏
        toolbar = NavigationToolbar2Tk(self.canvas, fig_frame)
        toolbar.update()

        # 初始绘图
        self.update_plot()

        # 启动动画
        self.animate()

    def create_controls(self, frame):
        """创建交互控件"""
        mu_label = tk.Label(frame, text="均值 (μ):")
        mu_label.grid(row=0, column=0, sticky=tk.W)

        self.mu_slider = tk.Scale(frame, from_=-10, to=10, orient=tk.HORIZONTAL,
                                 command=self.on_mu_change, length=300)
        self.mu_slider.set(self.mu)
        self.mu_slider.grid(row=0, column=1, sticky=tk.W)

        sigma_label = tk.Label(frame, text="标准差 (σ):")
        sigma_label.grid(row=1, column=0, sticky=tk.W)

        self.sigma_slider = tk.Scale(frame, from_=0.5, to=5, resolution=0.1,
                                   orient=tk.HORIZONTAL, command=self.on_sigma_change, length=300)
        self.sigma_slider.set(self.sigma)
        self.sigma_slider.grid(row=1, column=1, sticky=tk.W)

        color_label = tk.Label(frame, text="颜色:")
        color_label.grid(row=2, column=0, sticky=tk.W)

        self.color_var = tk.StringVar(value=self.colour)
        colors = ['red', 'green', 'blue', 'purple', 'orange']

        color_frame = tk.Frame(frame)
        color_frame.grid(row=2, column=1, sticky=tk.W)

        for color in colors:
            rb = tk.Radiobutton(color_frame, text=color, variable=self.color_var,
                               value=color, command=self.on_color_change)
            rb.pack(side=tk.LEFT, padx=5)

    def on_mu_change(self, value):
        """手动调节μ"""
        self.mu = float(value)
        self.auto_mu = self.mu

    def on_sigma_change(self, value):
        """手动调节σ"""
        self.sigma = float(value)
        self.auto_sigma = self.sigma

    def on_color_change(self):
        """改变颜色"""
        self.colour = self.color_var.get()

    def update_plot(self):
        """更新图形"""
        self.ax1.clear()
        self.ax2.clear()

        x_axis = np.linspace(-10, 10, 1000)

        self.ax1.plot(x_axis, stats.norm.pdf(x_axis, self.mu, self.sigma),
                     c=self.colour, linewidth=2)
        self.ax1.set_xlabel('X')
        self.ax1.set_ylabel('pdf')
        self.ax1.set_ylim(0, 1)
        self.ax1.set_xlim(-10, 10)
        self.ax1.set_title(f'概率密度函数 (μ={self.mu:.2f}, σ={self.sigma:.2f})', fontweight="bold")
        self.ax1.grid(True, alpha=0.3)

        self.ax2.plot(x_axis, stats.norm.cdf(x_axis, self.mu, self.sigma),
                     c=self.colour, linewidth=2)
        self.ax2.set_xlabel('X')
        self.ax2.set_ylabel('cdf')
        self.ax2.set_ylim(0, 1)
        self.ax2.set_xlim(-10, 10)
        self.ax2.set_title(f'累积分布函数 (μ={self.mu:.2f}, σ={self.sigma:.2f})', fontweight="bold")
        self.ax2.grid(True, alpha=0.3)

        self.fig.tight_layout(pad=3.0)
        self.canvas.draw()

    def animate(self):
        """自动动画"""
        # 更新自动参数
        self.auto_mu += self.auto_mu_direction
        self.auto_sigma += self.auto_sigma_direction

        # 碰到边界反弹
        if self.auto_mu > 5 or self.auto_mu < -5:
            self.auto_mu_direction *= -1
        if self.auto_sigma > 3 or self.auto_sigma < 0.5:
            self.auto_sigma_direction *= -1

        # 应用到当前
        self.mu = self.auto_mu
        self.sigma = self.auto_sigma

        # 同步滑块位置
        self.mu_slider.set(self.mu)
        self.sigma_slider.set(self.sigma)

        # 更新图形
        self.update_plot()

        # 每 100 毫秒调用一次自身
        self.root.after(100, self.animate)

# 启动
root = tk.Tk()
app = InteractiveAutoDistribution(root)
root.mainloop()
